# Explore Google Analytics Tracking Data

In [1]:
%load_ext autoreload
%autoreload 2

Import necessary Python modules

In [2]:
import os
import sys

import numpy as np
import pandas as pd

Get relative path to project root directory

In [3]:
PROJ_ROOT_DIR = os.path.join(os.pardir)
src_dir = os.path.join(PROJ_ROOT_DIR, "src")
sys.path.append(src_dir)

Import custom Python modules

In [4]:
%aimport bigquery_auth_helpers
from bigquery_auth_helpers import auth_to_bigquery

%aimport transform_helpers
import transform_helpers as th

## About

This step explores the raw Google Analytics (GA 360) tracking data in order to build up a BigQuery SQL query that returns
- attributes of the first visit to the store
- whether a purchase was made during a return visit to the store

BigQuery SQL is used to retrieve the following
1. how to extract the outcome of return visits
   - this is the label for machine learning
2. how to extract the number of products in a product list that were
   - viewed
   - clicked
3. etc.

## User Inputs

Define the following

1. start and end dates for train and test data to explore return visits
2. end dates for train and test data to explore clickthrough rate by product and visitor
3. list of visit attributes that define a unique visit

In [5]:
# 1. return visits
train_split_start_date = "20160901"
test_split_end_date = "20160930"

# 2. clickthrough rate
train_split_end_date_ctr = "20160902"

# 3. attributes that define a single visit
visit_definition_cols = [
    'fullvisitorid',
    'visitId',
    'visitNumber',
    'visitStartTime',
    'medium',
    'channelGrouping',
    'hits',
    'bounces',
    'pageviews',
    'time_on_site',
    'browser',
    'os',
    'deviceCategory',
]

::: {.content-hidden}
Get path to data sub-folders
:::

In [6]:
data_dir = os.path.join(PROJ_ROOT_DIR, "data")
gcp_keys_dir = os.path.join(PROJ_ROOT_DIR, "gcp_keys")

::: {.content-hidden}
Create a mapping between action type integer and label, in order to get meaningful names from the `action_type` column
:::

In [7]:
action_mapper = {
    1: "Click through of product lists",
    2: "Product detail views",
    3: "Add product(s) to cart",
    4: "Remove product(s) from cart",
    5: "Check out",
    6: "Completed purchase",
    7: "Refund of purchase",
    8: "Checkout options",
    0: "Unknown",
}

## Authenticate to `BigQuery`

In [8]:
gcp_auth_dict = auth_to_bigquery(gcp_keys_dir)

## Explore Google Analytics Tracking Data

### Get Return Visit Outcome

Get the outcome of all return visits during September 2016

In [9]:
%%time
query_str = f"""
            WITH
            return_visits AS (
                 SELECT fullvisitorid,
                        visitId,
                        visitNumber,
                        DATETIME(TIMESTAMP(TIMESTAMP_SECONDS(visitStartTime)), 'US/Pacific') AS visitStartTime,
                        IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, True, False) AS made_purchase_on_future_visit
                 FROM `data-to-insights.ecommerce.web_analytics`
                 WHERE date BETWEEN '{train_split_start_date}' AND '{test_split_end_date}'
                 AND geoNetwork.country = 'United States'
                 GROUP BY fullvisitorid,
                          visitId,
                          visitNumber,
                          visitStartTime
            )
            SELECT *
            FROM return_visits
            ORDER BY fullvisitorid
            """
df = th.extract_data(query_str, gcp_auth_dict)
df.head()

Query execution start time = 2023-07-02 12:43:55.233...done at 2023-07-02 12:43:58.086 (2.853 seconds).
Query returned 28,013 rows
CPU times: user 612 ms, sys: 34.4 ms, total: 646 ms
Wall time: 2.86 s


,fullvisitorid,visitId,visitNumber,visitStartTime,made_purchase_on_future_visit
0,0000093957001069502,1474985724,1,2016-09-27 07:15:24,False
1,0000245437374675368,1472862842,1,2016-09-02 17:34:02,False
2,0000639845445148063,1473694653,1,2016-09-12 08:37:33,False
3,000139156957304532,1473013369,1,2016-09-04 11:22:49,False
4,0001601342180848204,1474930306,1,2016-09-26 15:51:46,False


Get visitors who made multiple return visits

In [10]:
df_return_visits = df[df.duplicated(subset=["fullvisitorid"], keep=False)]
df_return_visits.head(10)

,fullvisitorid,visitId,visitNumber,visitStartTime,made_purchase_on_future_visit
12,0005884918507288420,1474464720,1,2016-09-21 06:32:00,False
13,0005884918507288420,1474469723,2,2016-09-21 07:55:23,False
19,0010286039787739137,1475084026,1,2016-09-28 10:33:46,False
20,0010286039787739137,1475249827,2,2016-09-30 08:37:07,False
32,0015065858137292339,1473641554,5,2016-09-11 17:52:34,False
33,0015065858137292339,1472856937,1,2016-09-02 15:55:37,False
34,0015065858137292339,1473997381,6,2016-09-15 20:43:01,False
35,0015065858137292339,1473119040,2,2016-09-05 16:44:00,False
36,0015065858137292339,1473630321,4,2016-09-11 14:45:21,False
37,0015065858137292339,1473466324,3,2016-09-09 17:12:04,False


Return visits are made up by two types of visitors

1. those that made a purchase
2. those that did not make a purchase

The number of visitors who did and did not make a purchase during a return visit are shown below

In [11]:
df_return_visits["made_purchase_on_future_visit"].value_counts().reset_index().assign(
    proportion=lambda df: df["count"] / df["count"].sum()
)

,made_purchase_on_future_visit,count,proportion
0,False,10507,0.955616
1,True,488,0.044384


::: {.callout-tip title="Observations"}

1. Only approximately 4.4% of all return visits in September 2016 resulted in a purchase.
:::

### Show Frequency of Repeat Customers During Return Visits

Some visitors made multiple return visits and a small subset of such visitors made multiple purchases. This is shown below.

First, for each visitor, get the following

1. whether that vistor ever made a purchase on a return visit
2. number of purchases made across all return visits

In [12]:
df_return_visits_with_purchase = df_return_visits.groupby(
    "fullvisitorid", as_index=False
).agg({"made_purchase_on_future_visit": ["max", "sum"]})
df_return_visits_with_purchase.columns = [
    "_".join(a).rstrip("_")
    for a in df_return_visits_with_purchase.columns.to_flat_index()
]
df_return_visits_with_purchase

,fullvisitorid,made_purchase_on_future_visit_max,made_purchase_on_future_visit_sum
0,0005884918507288420,False,0
1,0010286039787739137,False,0
2,0015065858137292339,False,0
3,0026203741366904270,True,1
4,0027817676806595220,False,0
...,...,...,...
3755,9986848664463401272,False,0
3756,9990362099175067703,False,0
3757,999203594099745000,False,0
3758,9992704342633956099,False,0


Next, get visitors who made a purchase on a return visit

In [13]:
df_return_visits_with_purchase = df_return_visits_with_purchase.query(
    "made_purchase_on_future_visit_max == True"
)
df_return_visits_with_purchase

,fullvisitorid,made_purchase_on_future_visit_max,made_purchase_on_future_visit_sum
3,0026203741366904270,True,1
7,0036417634769000138,True,1
8,0037518757923116572,True,1
14,0061519776091452595,True,1
19,0070976956518566605,True,1
...,...,...,...
3709,9891815404632176641,True,1
3718,9912185644936709935,True,1
3725,9941749289816017941,True,1
3742,9961396584113412108,True,1


::: {.callout-note title="Notes"}

1. `made_purchase_on_future_visit` indicates if a purchase was made during a return visit.
2. `made_purchase_on_future_visit_max` indicates whether a visitor who made a return visit to the store made a purchase during any such visit.
3. `made_purchase_on_future_visit_sum` shows the total number of purchases made by a visitor across all return visits.
:::

Finally, filter the occurrences of return visits during September 2016 to only capture all visitors who made **a purchase on multiple return visits**

In [14]:
df_return_visits_merged = df_return_visits.merge(
    df_return_visits_with_purchase, on=["fullvisitorid"], how="inner"
)

Visitors who made a purchase on multiple return visits are shown below

In [15]:
df_return_visits_merged.query("made_purchase_on_future_visit_sum > 2").head(15)

,fullvisitorid,visitId,visitNumber,visitStartTime,made_purchase_on_future_visit,made_purchase_on_future_visit_max,made_purchase_on_future_visit_sum
316,2074164338647079047,1472735454,16,2016-09-01 06:10:54,False,True,4
317,2074164338647079047,1474373863,23,2016-09-20 05:17:43,True,True,4
318,2074164338647079047,1473196307,19,2016-09-06 14:11:47,True,True,4
319,2074164338647079047,1472746638,18,2016-09-01 09:17:18,False,True,4
320,2074164338647079047,1472739238,17,2016-09-01 07:13:58,False,True,4
321,2074164338647079047,1473792585,21,2016-09-13 11:49:45,True,True,4
322,2074164338647079047,1473446196,20,2016-09-09 11:36:36,True,True,4
323,2074164338647079047,1474310682,22,2016-09-19 11:44:42,False,True,4
535,280738376597848400,1473873900,2,2016-09-14 10:25:00,True,True,3
536,280738376597848400,1473903584,5,2016-09-14 18:39:44,True,True,3


::: {.callout-tip title="Observations"}

1. We can see that these are the subset of return visitors who made a purchase on multiple return visits since the values in the `made_purchase_on_future_visit_sum` column that are larger than 1.
:::

For visitors who who made a purchae on multiple return visits, show the number of visitors who made a purchase on a single such visit and those that made a purchase during multiple such visits

In [16]:
df_return_visits_merged.assign(
    made_purchase_on_multiple_return_visits=lambda df: df[
        "made_purchase_on_future_visit_sum"
    ]
    > 1
).groupby("made_purchase_on_multiple_return_visits", as_index=False)[
    "fullvisitorid"
].count().rename(
    columns={"fullvisitorid": "count"}
).assign(
    proportion=lambda df: df["count"] / df["count"].sum()
)

,made_purchase_on_multiple_return_visits,count,proportion
0,False,1385,0.835344
1,True,273,0.164656


::: {.callout-tip title="Observations"}

1. Most visitors who made a purchae on multiple return visits during September 2016 only made a purchase during one of thse visits. Only approximately 16% of such visitors made a purchase during multiple such visits.
2. The SQL logic defined above is capturing repeat customers. These are visitors who made a purchase during more than one visit to the store. This is required per the scope of this project since the business wants to grow both repeat as well as new customers.
:::

### Product List Clickthrough Rate

Get the following product-related actions performed per visit
- `productListName`
- `productListPosition`
- `isImpression`
- `isClick`

In [17]:
query_str = f"""
            WITH
            -- Step 1. get visitors with a return visit
            returning_visitors AS (
                 SELECT fullvisitorid,
                        IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, True, False) AS made_purchase_on_future_visit
                 FROM `data-to-insights.ecommerce.web_analytics`
                 WHERE date BETWEEN '{train_split_start_date}' AND '{test_split_end_date}'
                 AND geoNetwork.country = 'United States'
                 GROUP BY fullvisitorid
            ),
            -- Steps 2. and 3. get attributes of the first visit
            first_visit_attributes AS (
                SELECT -- =========== GEOSPATIAL AND TEMPORAL ATTRIBUTES OF VISIT ===========
                       geoNetwork.country,
                       -- =========== VISIT AND VISITOR METADATA ===========
                       fullvisitorid,
                       visitId,
                       visitNumber,
                       DATETIME(TIMESTAMP(TIMESTAMP_SECONDS(visitStartTime)), 'US/Pacific') AS visitStartTime,
                       -- =========== SOURCE OF SITE TRAFFIC ===========
                       -- source of the traffic from which the visit was initiated
                       trafficSource.source,
                       -- medium of the traffic from which the visit was initiated
                       trafficSource.medium,
                       -- referring channel connected to visit
                       channelGrouping,
                       -- =========== VISITOR ACTIVITY ===========
                       -- total number of hits
                       (CASE WHEN totals.hits > 0 THEN totals.hits ELSE 0 END) AS hits,
                       -- number of bounces
                       -- action performed during first visit
                       CAST(h.eCommerceAction.action_type AS INT64) AS action_type,
                       (CASE WHEN totals.bounces > 0 THEN totals.bounces ELSE 0 END) AS bounces,
                       -- page views
                       IFNULL(totals.pageviews, 0) AS pageviews,
                       -- time on the website
                       IFNULL(totals.timeOnSite, 0) AS time_on_site,
                       -- =========== VISITOR DEVICES ===========
                       -- user's browser
                       device.browser,
                       -- user's operating system
                       device.operatingSystem AS os,
                       -- user's type of device
                       device.deviceCategory,
                       -- =========== PRODUCT ===========
                       h.product,
                       -- =========== ML LABEL (DEPENDENT VARIABLE) ===========
                       made_purchase_on_future_visit
                FROM `data-to-insights.ecommerce.web_analytics`,
                UNNEST(hits) AS h
                INNER JOIN returning_visitors USING (fullvisitorid)
                WHERE date BETWEEN '{train_split_start_date}' AND '{train_split_end_date_ctr}'
                AND geoNetwork.country = 'United States'
                AND totals.newVisits = 1
            )
            SELECT fullvisitorid,
                   visitId,
                   visitNumber,
                   visitStartTime,
                   medium,
                   channelGrouping,
                   hits,
                   bounces,
                   action_type,
                   pageviews,
                   time_on_site,
                   browser,
                   os,
                   deviceCategory,
                   pu.productListName,
                   pu.productListPosition,
                   pu.isImpression,
                   pu.isClick
            FROM first_visit_attributes
            LEFT JOIN UNNEST(product) as pu
            """
df_actions = (
    th.extract_data(query_str, gcp_auth_dict)
    .assign(action_type=lambda df: df['action_type'].map(action_mapper))
)
df_actions

Query execution start time = 2023-07-02 12:43:58.244...done at 2023-07-02 12:44:09.040 (10.796 seconds).
Query returned 110,949 rows


,fullvisitorid,visitId,visitNumber,visitStartTime,medium,channelGrouping,hits,bounces,action_type,pageviews,time_on_site,browser,os,deviceCategory,productListName,productListPosition,isImpression,isClick
0,2787909377637066215,1472780448,1,2016-09-01 18:40:48,organic,Organic Search,5,0,Unknown,5,283,Chrome,Macintosh,desktop,None,<NA>,<NA>,<NA>
1,2787909377637066215,1472780448,1,2016-09-01 18:40:48,organic,Organic Search,5,0,Unknown,5,283,Chrome,Macintosh,desktop,None,<NA>,<NA>,<NA>
2,2787909377637066215,1472780448,1,2016-09-01 18:40:48,organic,Organic Search,5,0,Unknown,5,283,Chrome,Macintosh,desktop,None,<NA>,<NA>,<NA>
3,2787909377637066215,1472780448,1,2016-09-01 18:40:48,organic,Organic Search,5,0,Unknown,5,283,Chrome,Macintosh,desktop,None,<NA>,<NA>,<NA>
4,2787909377637066215,1472780448,1,2016-09-01 18:40:48,organic,Organic Search,5,0,Unknown,5,283,Chrome,Macintosh,desktop,None,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110944,639074571300803633,1472795110,1,2016-09-01 22:45:10,organic,Organic Search,4,0,Unknown,4,151,Chrome,Windows,desktop,Category,17,True,<NA>
110945,639074571300803633,1472795110,1,2016-09-01 22:45:10,organic,Organic Search,4,0,Unknown,4,151,Chrome,Windows,desktop,Category,18,True,<NA>
110946,639074571300803633,1472795110,1,2016-09-01 22:45:10,organic,Organic Search,4,0,Unknown,4,151,Chrome,Windows,desktop,Category,19,True,<NA>
110947,639074571300803633,1472795110,1,2016-09-01 22:45:10,organic,Organic Search,4,0,Unknown,4,151,Chrome,Windows,desktop,Category,20,True,<NA>


Show the number of unique values in the `isImpression` and `isClick` columns

In [18]:
for c in ['isImpression', 'isClick']:
    print(c, df_actions[c].unique().tolist())

isImpression [<NA>, True]
isClick [<NA>, True]


Show all visits with a product click

In [19]:
display(df_actions.query("isClick == True"))

,fullvisitorid,visitId,visitNumber,visitStartTime,medium,channelGrouping,hits,bounces,action_type,pageviews,time_on_site,browser,os,deviceCategory,productListName,productListPosition,isImpression,isClick
88,6055189100224767003,1472755933,1,2016-09-01 11:52:13,cpm,Display,5,0,Click through of product lists,4,17,Chrome,Macintosh,desktop,Category,0,<NA>,True
968,7109081858569729237,1472755255,1,2016-09-01 11:40:55,cpm,Display,9,0,Click through of product lists,8,82,Chrome,Macintosh,desktop,Search Results,0,<NA>,True
1094,6495012494710721260,1472828880,1,2016-09-02 08:08:00,cpc,Paid Search,10,0,Click through of product lists,8,262,Chrome,Windows,desktop,Category,0,<NA>,True
1095,6495012494710721260,1472828880,1,2016-09-02 08:08:00,cpc,Paid Search,10,0,Click through of product lists,8,262,Chrome,Windows,desktop,Category,23,<NA>,True
1443,9275813847347927121,1472882823,1,2016-09-02 23:07:03,cpc,Paid Search,10,0,Click through of product lists,7,53,Chrome,Macintosh,desktop,Category,0,<NA>,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109743,8486302275562434243,1472746894,1,2016-09-01 09:21:34,(none),Direct,67,0,Click through of product lists,46,1298,Chrome,Windows,desktop,Category,31,<NA>,True
109782,8486302275562434243,1472746894,1,2016-09-01 09:21:34,(none),Direct,67,0,Click through of product lists,46,1298,Chrome,Windows,desktop,Category,0,<NA>,True
109784,8486302275562434243,1472746894,1,2016-09-01 09:21:34,(none),Direct,67,0,Click through of product lists,46,1298,Chrome,Windows,desktop,Category,13,<NA>,True
110589,093437836654213633,1472857366,1,2016-09-02 16:02:46,organic,Organic Search,3,0,Click through of product lists,2,18,Safari,iOS,mobile,Category,0,<NA>,True


Get two visits with a product click

In [20]:
visits_with_click = df_actions.query("isClick == True")['visitId'].unique().tolist()
visitIds_clicked = np.random.choice(visits_with_click, size=2, replace=False).tolist()

Show two visits with a product click

In [21]:
with pd.option_context('display.max_rows', None):
    display(df_actions.query("visitId.isin(@visitIds_clicked)"))

,fullvisitorid,visitId,visitNumber,visitStartTime,medium,channelGrouping,hits,bounces,action_type,pageviews,time_on_site,browser,os,deviceCategory,productListName,productListPosition,isImpression,isClick
19597,569579103002599546,1472875116,1,2016-09-02 20:58:36,referral,Social,12,0,Unknown,8,153,Chrome,Macintosh,desktop,None,<NA>,<NA>,<NA>
19598,569579103002599546,1472875116,1,2016-09-02 20:58:36,referral,Social,12,0,Unknown,8,153,Chrome,Macintosh,desktop,Category,1,True,<NA>
19599,569579103002599546,1472875116,1,2016-09-02 20:58:36,referral,Social,12,0,Unknown,8,153,Chrome,Macintosh,desktop,Category,2,True,<NA>
19600,569579103002599546,1472875116,1,2016-09-02 20:58:36,referral,Social,12,0,Unknown,8,153,Chrome,Macintosh,desktop,Category,3,True,<NA>
19601,569579103002599546,1472875116,1,2016-09-02 20:58:36,referral,Social,12,0,Unknown,8,153,Chrome,Macintosh,desktop,Category,4,True,<NA>
19602,569579103002599546,1472875116,1,2016-09-02 20:58:36,referral,Social,12,0,Unknown,8,153,Chrome,Macintosh,desktop,Category,5,True,<NA>
19603,569579103002599546,1472875116,1,2016-09-02 20:58:36,referral,Social,12,0,Unknown,8,153,Chrome,Macintosh,desktop,Category,6,True,<NA>
19604,569579103002599546,1472875116,1,2016-09-02 20:58:36,referral,Social,12,0,Unknown,8,153,Chrome,Macintosh,desktop,Category,7,True,<NA>
19605,569579103002599546,1472875116,1,2016-09-02 20:58:36,referral,Social,12,0,Unknown,8,153,Chrome,Macintosh,desktop,Category,8,True,<NA>
19606,569579103002599546,1472875116,1,2016-09-02 20:58:36,referral,Social,12,0,Unknown,8,153,Chrome,Macintosh,desktop,Category,9,True,<NA>


Product clicks are not observed for the `Unknown` action type, as shown below

In [22]:
assert df_actions.query("action_type == 'Unknown'").query("isClick == True").empty

Calculate the product clickthrough rate (CTR) for all products [in a product list](https://metriclabs.com.au/glossary/analytics-metrics/product-list-views/) across these two visits

In [23]:
(
    df_actions.query("visitId.isin(@visitIds_clicked)")
    .groupby("visitId", as_index=False)
    .agg({'isImpression': 'sum', 'isClick': 'sum'})
    .rename(columns={"isClick": "clicks", "isImpression": "views"})
    .assign(product_ctr=lambda df: 100*(df['clicks']/df['views']))
)

,visitId,views,clicks,product_ctr
0,1472851024,453,7,1.545254
1,1472875116,80,3,3.75


Using Python, calculate the clickthrough rate (CTR) for all products in a product list across all visits
- by product
  - number of products that were clicked in a product list / number of products that were viewed a product list
- by visitor
  - number of visitors who clicked a product in a product list / number of visitors who viewed a product list

In [31]:
df_summary_actions = (
    pd.DataFrame.from_dict(
        {
            "num_visitors": df_actions["fullvisitorid"].nunique(),
            "actions": len(df_actions.query("action_type != 'Unknown'")),
            # views by product
            "product_views": df_actions["isImpression"].sum(),
            # clicks by product
            "product_clicks": df_actions["isClick"].sum(),
            # views by visitor
            'visitor_views': (
                len(
                    df_actions.query("(action_type != 'Click through of product lists') & (isImpression == True)")
                    .drop_duplicates(subset=visit_definition_cols, keep='first')
                )
            ),
            # clicks by visitor
            'visitor_clicks': (
                len(
                    df_actions.query("(action_type == 'Click through of product lists') & (isClick == True)")
                    .drop_duplicates(subset=visit_definition_cols, keep='first')
                )
            )
        },
        orient='index',
    )
    .transpose()
    # CTR by product
    .assign(product_ctr=lambda df: 100*(df['product_clicks']/df['product_views']))
    # CTR by visitor
    .assign(visitor_ctr=lambda df: 100*(df['visitor_clicks']/df['visitor_views']))
)
df_summary_actions

,num_visitors,actions,product_views,product_clicks,visitor_views,visitor_clicks,product_ctr,visitor_ctr
0,1359,4104,103119,1717,911,379,1.665067,41.602634


Get the following aggregated product-related actions performed per visit
- product views
- product clicks

In [39]:
query_str = f"""
            WITH
            -- Step 1. get visitors with a return visit
            returning_visitors AS (
                 SELECT fullvisitorid,
                        IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, True, False) AS made_purchase_on_future_visit
                 FROM `data-to-insights.ecommerce.web_analytics`
                 WHERE date BETWEEN '{train_split_start_date}' AND '{test_split_end_date}'
                 AND geoNetwork.country = 'United States'
                 GROUP BY fullvisitorid
            ),
            -- Steps 2. and 3. get attributes of the first visit
            first_visit_attributes AS (
                SELECT -- =========== GEOSPATIAL AND TEMPORAL ATTRIBUTES OF VISIT ===========
                       geoNetwork.country,
                       -- =========== VISIT AND VISITOR METADATA ===========
                       fullvisitorid,
                       visitId,
                       visitNumber,
                       DATETIME(TIMESTAMP(TIMESTAMP_SECONDS(visitStartTime)), 'US/Pacific') AS visitStartTime,
                       -- =========== SOURCE OF SITE TRAFFIC ===========
                       -- source of the traffic from which the visit was initiated
                       trafficSource.source,
                       -- medium of the traffic from which the visit was initiated
                       trafficSource.medium,
                       -- referring channel connected to visit
                       channelGrouping,
                       -- =========== VISITOR ACTIVITY ===========
                       -- total number of hits
                       (CASE WHEN totals.hits > 0 THEN totals.hits ELSE 0 END) AS hits,
                       -- number of bounces
                       -- action performed during first visit
                       CAST(h.eCommerceAction.action_type AS INT64) AS action_type,
                       (CASE WHEN totals.bounces > 0 THEN totals.bounces ELSE 0 END) AS bounces,
                       -- page views
                       IFNULL(totals.pageviews, 0) AS pageviews,
                       -- time on the website
                       IFNULL(totals.timeOnSite, 0) AS time_on_site,
                       -- =========== VISITOR DEVICES ===========
                       -- user's browser
                       device.browser,
                       -- user's operating system
                       device.operatingSystem AS os,
                       -- user's type of device
                       device.deviceCategory,
                       -- =========== PRODUCT ===========
                       h.product,
                       -- =========== ML LABEL (DEPENDENT VARIABLE) ===========
                       made_purchase_on_future_visit
                FROM `data-to-insights.ecommerce.web_analytics`,
                UNNEST(hits) AS h
                INNER JOIN returning_visitors USING (fullvisitorid)
                WHERE date BETWEEN '{train_split_start_date}' AND '{train_split_end_date_ctr}'
                AND geoNetwork.country = 'United States'
                AND totals.newVisits = 1
            ),
            -- Step 4. get aggregated features (attributes) per visit
            visit_attributes AS (
                SELECT fullvisitorid,
                       visitId,
                       visitNumber,
                       visitStartTime,
                       medium,
                       channelGrouping,
                       hits,
                       bounces,
                       MAX(action_type) AS last_action,
                       pageviews,
                       time_on_site,
                       browser,
                       os,
                       deviceCategory,
                       -- get number of products displayed and clicked during the first visit
                       -- (option 1) USING SUM()
                       -- SUM(CASE WHEN pu.isImpression IS TRUE THEN 1 ELSE 0 END) AS product_views,
                       -- SUM(CASE WHEN pu.isClick IS TRUE THEN 1 ELSE 0 END) AS product_clicks,
                       -- (option 2) USING COUNT()
                       COUNT(CASE WHEN pu.isImpression IS NULL THEN NULL ELSE 1 END) AS product_views,
                       COUNT(CASE WHEN pu.isClick IS NULL THEN NULL ELSE 1 END) AS product_clicks
                FROM first_visit_attributes
                LEFT JOIN UNNEST(product) as pu
                GROUP BY fullvisitorid,
                         visitId,
                         visitNumber,
                         visitStartTime,
                         medium,
                         channelGrouping,
                         hits,
                         bounces,
                         pageviews,
                         time_on_site,
                         browser,
                         os,
                         deviceCategory
            )
            SELECT *
            FROM visit_attributes
            """
df_visitors = (
    th.extract_data(query_str, gcp_auth_dict)
    .assign(last_action=lambda df: df['last_action'].map(action_mapper))
)
df_visitors

Query execution start time = 2023-07-02 13:43:56.176...done at 2023-07-02 13:43:58.144 (1.968 seconds).
Query returned 1,360 rows


,fullvisitorid,visitId,visitNumber,visitStartTime,medium,channelGrouping,hits,bounces,last_action,pageviews,time_on_site,browser,os,deviceCategory,product_views,product_clicks
0,9750540744658016166,1472748832,1,2016-09-01 09:53:52,organic,Organic Search,39,0,Product detail views,17,1334,Chrome,Android,mobile,188,4
1,5133217183540194806,1472877510,1,2016-09-02 21:38:30,organic,Organic Search,19,0,Add product(s) to cart,13,338,Safari,Macintosh,desktop,133,4
2,7370259867769888657,1472856605,1,2016-09-02 15:50:05,organic,Organic Search,4,0,Unknown,3,37,Chrome,Android,mobile,34,0
3,3108089013033989361,1472767301,1,2016-09-01 15:01:41,organic,Organic Search,4,0,Unknown,3,24,Chrome,Android,mobile,34,0
4,3497940039722152076,1472869927,1,2016-09-02 19:32:07,organic,Organic Search,4,0,Unknown,3,36,Safari,iOS,tablet,34,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1355,9609367222527813362,1472844552,1,2016-09-02 12:29:12,organic,Organic Search,1,1,Unknown,1,0,Chrome,Windows,desktop,34,0
1356,185444327419942710,1472758915,1,2016-09-01 12:41:55,referral,Social,1,1,Unknown,1,0,Chrome,Windows,desktop,21,0
1357,664300994101851701,1472771026,1,2016-09-01 16:03:46,(none),Direct,2,0,Unknown,2,74,Chrome,Macintosh,desktop,25,0
1358,7567453673632648637,1472868432,1,2016-09-02 19:07:12,referral,Social,2,0,Unknown,2,27,Safari (in-app),iOS,mobile,46,0


::: {.callout-note title="Notes"}

1. From the [datset documentation](https://support.google.com/analytics/answer/3437719?hl=en) for the **Field Name** `hits.eCommerceAction.action_type`, in order to calculate number of products in a product list it is sufficient to use the SQL conditional `WHERE hits.product.isImpression == TRUE`. This was used as part of a `CASE WHEN` in option 1 with the `SUM()` aggregation function to count the number of products in a product list that were viewed. Alternatively, since the SQL [`COUNT()` function counts non-`NULL` values](https://stackoverflow.com/a/1354071/4057186), this function could also be used to get the same result, as shown in the two commented lines in option 2.
2. Similar logic for product views is also used to get the number of products in a product list that were clicked.
:::

Repeat the Python calculation of the clickthrough rate (CTR) for all products in a product list across all visits
- by product
- by visitor

using the aggregated visits data retrieved, where the calculation is performed using SQL

In [40]:
df_summary_actions_sql = (
    pd.DataFrame.from_dict(
        {
            "num_visitors": df_visitors["fullvisitorid"].nunique(),
            # views by product
            "product_views": df_visitors['product_views'].sum(),
            # clicks by product
            "product_clicks": df_visitors['product_clicks'].sum(),
            # views by visitor
            "visitor_views": (
                len(
                    df_visitors.query("product_views > 0")
                    .drop_duplicates(subset=visit_definition_cols, keep='first')
                )
            ),
            # clicks by visitor
            "visitor_clicks": (
                len(
                    df_visitors.query("product_clicks > 0")
                    .drop_duplicates(subset=visit_definition_cols, keep='first')
                )
            ),
        },
        orient='index',
    )
    .transpose()
    # CTR by product
    .assign(product_ctr=lambda df: 100*(df['product_clicks']/df['product_views']))
    # CTR by visitor
    .assign(visitor_ctr=lambda df: 100*(df['visitor_clicks']/df['visitor_views']))
)
df_summary_actions_sql

,num_visitors,product_views,product_clicks,visitor_views,visitor_clicks,product_ctr,visitor_ctr
0,1359,103119,1717,911,379,1.665067,41.602634


Verify that the CTR by products and visitors calculated using visits aggregated with
- Python
- SQL

are equivalent

In [30]:
assert df_summary_actions.drop(columns=['actions']).equals(df_summary_actions_sql)